<a href="https://colab.research.google.com/github/bannis14/Portfolio-Projects/blob/main/Time_Series_Forecast/RNN_LSTM_Model_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN_LSTM Model Forecasting

## Libraries and Data
  *     Need to Connect to GPU



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Time Series Forecasting Product

/content/drive/MyDrive/Time Series Forecasting Product


In [ ]:
!pip install pyyaml
!pip install darts

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import ParameterGrid
from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel
from sklearn.metrics import mean_squared_error

In [ ]:
# Load the Data
df = pd.read_csv('/content/drive/MyDrive/Time Series Forecasting Product/nyc_data.csv', index_col=0, parse_dates=True)
future_df = pd.read_csv('/content/drive/MyDrive/Time Series Forecasting Product/future.csv', index_col=0, parse_dates=True)
df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [ ]:
# Extract Regressors
X_train = df.iloc[:, 1:]
X_future = future_df.iloc[:, 1:]

In [ ]:
X = pd.concat([X_train, X_future])

In [ ]:
# Rename Variables
df = df.rename(columns={'Demand': 'y'})
df.head(0)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,


## Prep for LSTM


In [ ]:
# Time Series Object
series = TimeSeries.from_series(df.y)
covariates = TimeSeries.from_dataframe(X)

In [ ]:
# year
year_series = datetime_attribute_timeseries(
    pd.date_range(start=series.start_time(),
                  freq=series.freq_str,
                  periods=X.shape[0]),
    attribute="year",
    one_hot=False)

In [ ]:
# Month
month_series = datetime_attribute_timeseries(year_series,
                                             attribute="month",
                                             one_hot=True)

In [ ]:
# Weekday
weekday_series = datetime_attribute_timeseries(year_series,
                                               attribute="weekday",
                                               one_hot=True)

In [ ]:
# Prepping the Scaler
transformer1 = Scaler()
transformer2 = Scaler()

In [ ]:
y_transformed = transformer1.fit_transform(series)

In [ ]:
covariates = covariates.stack(year_series)
covariates_transformed = transformer2.fit_transform(covariates)
covariates_transformed = covariates_transformed.stack(month_series)
covariates_transformed = covariates_transformed.stack(weekday_series)

## LSTM Model


In [ ]:
# get the best parameters
parameters = pd.read_csv("Forecasting Product/best_params_lstm.csv",
                         index_col=0)
parameters

,12
dropout,0.000000
hidden_dim,20.000000
input_chunk_length,20.000000
lr,0.003000
n_epochs,10.000000
n_rnn_layers,2.000000
training_length,20.000000
rmse,108.677799


In [ ]:
# Getting the parameters

n_rnn_layers = int(parameters.loc["n_rnn_layers"][0])
dropout = float(parameters.loc["dropout"][0])
hidden_dim = int(parameters.loc["hidden_dim"][0])
input_chunk_length = int(parameters.loc["input_chunk_length"][0])
lr = float(parameters.loc["lr"][0])
n_epochs = int(parameters.loc["n_epochs"][0])
training_length = int(parameters.loc["training_length"][0])

In [ ]:
# Model
model = RNNModel("LSTM",
                 hidden_dim=hidden_dim,
                 n_rnn_layers=n_rnn_layers,
                 dropout=dropout,
                 n_epochs=n_epochs,
                 optimizer_kwargs={'lr': lr},
                 random_state=1502,
                 training_length=training_length,
                 input_chunk_length=input_chunk_length,
                 pl_trainer_kwargs={"accelerator": "gpu", "devices": [0]},
                 save_checkpoints= True,
                 )

In [ ]:
# Fit the model to the data
model.fit(y_transformed,
          future_covariates=covariates_transformed)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | RNN              | 1.8 K 
4 | V             | Linear           | 21    
---------------------------------------------------
1.8 K     Trainable params
0         Non-trainable params
1.8 K     Total params
0.007     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:369: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['train_loss', 'epoch', 'step']. HINT: Did you call `log('val_loss', value)` in the `LightningModule`?
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


RNNModel(input_chunk_length=20, model=RNN, hidden_dim=20, n_rnn_layers=2, dropout=0.0, training_length=20, n_epochs=10, optimizer_kwargs={'lr': 0.003}, random_state=1502, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': [0]}, save_checkpoints=True)

## Predictions


In [ ]:
predictions_lstm = model.predict(n=len(future_df),
                                 future_covariates=covariates_transformed)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
predictions_lstm = TimeSeries.pd_series(
    transformer1.inverse_transform(predictions_lstm)).rename("lstm")

In [ ]:
predictions_lstm

Date
2021-01-01     716.401502
2021-01-02     875.108866
2021-01-03     777.071181
2021-01-04     919.030032
2021-01-05     781.219549
2021-01-06     967.334989
2021-01-07     853.743915
2021-01-08     746.668161
2021-01-09     913.587348
2021-01-10     821.889788
2021-01-11     960.668854
2021-01-12     809.999431
2021-01-13    1005.186813
2021-01-14     889.842097
2021-01-15     777.641919
2021-01-16     948.198202
2021-01-17     852.848006
2021-01-18     989.082697
2021-01-19     852.249612
2021-01-20    1033.732063
2021-01-21     918.761716
2021-01-22     804.556905
2021-01-23     961.727372
2021-01-24     871.248426
2021-01-25    1003.934475
2021-01-26     865.748074
2021-01-27    1053.660994
2021-01-28     939.041243
2021-01-29     834.901417
2021-01-30    1007.445092
2021-01-31     910.756975
Freq: D, Name: lstm, dtype: float64

In [ ]:
predictions_lstm.to_csv("Forecasting Product/Ensemble/predictions_lstm.csv")